# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,3.81,93,0,1.54,AR,1731558868
1,1,grytviken,-54.2811,-36.5092,-1.59,81,71,2.02,GS,1731559098
2,2,hawaiian paradise park,19.5933,-154.9731,22.73,97,100,3.09,US,1731559099
3,3,margaret river,-33.9500,115.0667,22.47,55,32,7.48,AU,1731559101
4,4,saint-felicien,48.6501,-72.4491,-6.62,86,0,2.06,CA,1731559102


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [25]:
%%capture --no-display

map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM", 
    size = 'Humidity',
    scale = .75,
    color = "City",
    alpha = .5,
    frame_width = 800,
    frame_height = 500
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

##### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
prefered_df = city_data_df.loc[(city_data_df["Max Temp"] <= 32) & (
    city_data_df["Max Temp"] >= 25), :]

prefered_df = prefered_df.loc[(city_data_df["Cloudiness"] <= 50), :]

# Drop any rows with null values
prefered_df.dropna()

# Display sample data
prefered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,fale old settlement,-9.3852,-171.2468,28.61,78,36,7.56,TK,1731559113
31,31,west island,-12.1568,96.8225,28.99,74,40,7.72,CC,1731559134
42,42,uttar char fasson,22.2332,90.7132,27.06,58,0,5.75,BD,1731559146
53,53,utrik,11.2278,169.8474,28.13,72,9,6.22,MH,1731559159
63,63,nungwi,-5.7265,39.2987,27.06,78,9,1.28,TZ,1731559171
...,...,...,...,...,...,...,...,...,...,...
554,554,myeik,12.4333,98.6000,29.69,65,0,1.37,MM,1731559736
555,555,lunavada,23.1333,73.6167,29.13,31,1,3.25,IN,1731559737
557,557,mangrol,21.1167,70.1167,29.22,42,18,4.28,IN,1731559739
559,559,bogale,16.2833,95.4000,29.36,74,0,3.64,MM,1731559741


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = prefered_df.copy()[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
13,fale old settlement,TK,-9.3852,-171.2468,78,
31,west island,CC,-12.1568,96.8225,74,
42,uttar char fasson,BD,22.2332,90.7132,58,
53,utrik,MH,11.2278,169.8474,72,
63,nungwi,TZ,-5.7265,39.2987,78,
...,...,...,...,...,...,...
554,myeik,MM,12.4333,98.6000,65,
555,lunavada,IN,23.1333,73.6167,31,
557,mangrol,IN,21.1167,70.1167,42,
559,bogale,MM,16.2833,95.4000,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
limit = 20

params = {
    "limit":limit,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    params["categories"] = f"accommodation.hotel"
        
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fale old settlement - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
uttar char fasson - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
nungwi - nearest hotel: Mama Fatuma
nishinoomote - nearest hotel: ゼウスハウス
al jaradiyah - nearest hotel: No hotel found
carutapera - nearest hotel: No hotel found
road town - nearest hotel: No hotel found
axim - nearest hotel: Axim Beach Hotel
atafu village - nearest hotel: No hotel found
bengkulu - nearest hotel: Dena Hotel
rongelap - nearest hotel: No hotel found
tolanaro - nearest hotel: Hôtel Mahavokey
georgetown - nearest hotel: Page 63 hostel
tabou - nearest hotel: hôtel le rochet
diu - nearest hotel: Hotel Ankur
chuhar jamali - nearest hotel: No hotel found
madimba - nearest hotel: No hotel found
porto novo - nearest hotel: Hôtel 6500 CFA
broken hill - nearest hotel: Ibis Styles
mukalla - nearest hotel: فندق الجابري
labuan - nearest hotel: Labuan Avenue Hotel
takoradi -

,City,Country,Lat,Lng,Humidity,Hotel Name
13,fale old settlement,TK,-9.3852,-171.2468,78,No hotel found
31,west island,CC,-12.1568,96.8225,74,Cocos Village Bungalows
42,uttar char fasson,BD,22.2332,90.7132,58,No hotel found
53,utrik,MH,11.2278,169.8474,72,No hotel found
63,nungwi,TZ,-5.7265,39.2987,78,Mama Fatuma
...,...,...,...,...,...,...
554,myeik,MM,12.4333,98.6000,65,Green Eyes Hotel
555,lunavada,IN,23.1333,73.6167,31,No hotel found
557,mangrol,IN,21.1167,70.1167,42,No hotel found
559,bogale,MM,16.2833,95.4000,74,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found", :]
hotel_df.count()

City          43
Country       43
Lat           43
Lng           43
Humidity      43
Hotel Name    43
dtype: int64

In [37]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM", 
    size = 'Humidity',
    scale = 1,
    color = "City",
    alpha = .5,
    frame_width = 800,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)